In [1]:
#import dependencies
import keras
from keras.datasets import imdb
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout, Embedding, Conv1D, GlobalMaxPool1D, SpatialDropout1D
from keras.callbacks import ModelCheckpoint
import os
from sklearn.metrics import roc_auc_score, roc_curve
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#set hyperparameters
output_dir = 'model_output/conv'

#training
epochs = 4
batch_size = 128

#vector-space embedding
n_dim = 64
n_unique_words = 5000
max_review_length = 400
pad_type = trunc_type = 'pre'
drop_embed = 0.2

#convolutional layer architecture
n_conv = 256 #filters, aka kernels
k_conv = 3 #kernel length

#dense layer architecture
n_dense = 256
dropout = 0.2

In [3]:
#model architecture
model = Sequential()

#vector-space embedding:
model.add(Embedding(n_unique_words, n_dim,
                    input_length=max_review_length))
model.add(SpatialDropout1D(drop_embed))

#convolutional layer:
model.add(Conv1D(n_conv, k_conv, activation='relu'))
#model.add(Conv1D(n_conv, k_conv, activation='relu'))
model.add(GlobalMaxPool1D())

#dense layer
model.add(Dense(n_dense, activation='relu'))
model.add(Dropout(dropout))

#output layer
model.add(Dense(1, activation='sigmoid'))

In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 400, 64)           320000    
                                                                 
 spatial_dropout1d (SpatialD  (None, 400, 64)          0         
 ropout1D)                                                       
                                                                 
 conv1d (Conv1D)             (None, 398, 256)          49408     
                                                                 
 global_max_pooling1d (Globa  (None, 256)              0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 256)               65792     
                                                                 
 dropout (Dropout)           (None, 256)               0

In [5]:
#data preprocessing
(x_train, y_train), (x_valid, y_valid) = imdb.load_data(num_words=n_unique_words)
#standardizing input length by padding and truncating
x_train = pad_sequences(x_train, maxlen=max_review_length, padding=pad_type, truncating=trunc_type, value=0)
x_valid = pad_sequences(x_valid, maxlen=max_review_length, padding=pad_type, truncating=trunc_type, value=0)

In [6]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [7]:
#Create an object and directory for checkpoint model parameters after each epoch
modelcheckpoint = ModelCheckpoint(filepath=output_dir+"/CNNweights.{epoch:02d}.hdf5")

if not os.path.exists((output_dir)):
    os.makedirs(output_dir)

In [8]:
model.fit(x_train, y_train,
          batch_size=batch_size, epochs=epochs, verbose=1,
          validation_data=(x_valid, y_valid),
          callbacks=[modelcheckpoint])

Epoch 1/4
196/196 [==============================] - 20s 101ms/step - loss: 0.4921 - accuracy: 0.7420 - val_loss: 0.2980 - val_accuracy: 0.8752
Epoch 2/4
196/196 [==============================] - 20s 102ms/step - loss: 0.2507 - accuracy: 0.9001 - val_loss: 0.2542 - val_accuracy: 0.8930
Epoch 3/4
196/196 [==============================] - 20s 104ms/step - loss: 0.1706 - accuracy: 0.9363 - val_loss: 0.2615 - val_accuracy: 0.8939
Epoch 4/4
196/196 [==============================] - 21s 105ms/step - loss: 0.1124 - accuracy: 0.9609 - val_loss: 0.2891 - val_accuracy: 0.8904
